In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('merged_data.xlsx')

In [3]:
df.describe()

,Project Code Parent,Invoiced Date,Actual Project Revenue,Complete,CPI,IR,LOI,Amount
count,517.000000,517,2422.000000,2422.000000,2422.000000,2422.000000,2422.000000,1905.000000
mean,20227.375242,2024-05-14 03:53:57.911025152,4605.728534,472.137077,17.698233,36.035582,14.960512,10460.770472
min,16874.000000,2024-01-02 00:00:00,0.000000,1.000000,1.000000,0.330000,1.000000,1.000000
25%,19629.000000,2024-03-18 00:00:00,1550.062500,107.500000,5.500000,15.000000,10.262500,3570.000000
50%,20231.000000,2024-05-14 00:00:00,3390.750000,300.000000,8.805000,30.000000,15.000000,8300.000000
75%,20938.000000,2024-06-30 00:00:00,6500.000000,604.500000,30.000000,50.000000,18.522500,15025.000000
max,21786.000000,2024-09-11 00:00:00,19500.000000,2273.000000,92.000000,100.000000,30.120000,40200.000000
std,905.097411,NaN,4086.928747,475.403948,17.490211,25.271899,5.134275,8761.397877


In [4]:
df.dtypes

,0
Project Code Parent,float64
Project Name,object
Invoiced Date,datetime64[ns]
Actual Project Revenue,float64
Client Name,object
Complete,int64
CPI,float64
IR,float64
LOI,float64
Countries,object


In [5]:
df.head(3)

,Project Code Parent,Project Name,Invoiced Date,Actual Project Revenue,Client Name,Complete,CPI,IR,LOI,Countries,Audience,Status,Deal Name,Amount,Client Segment Type
0,19014.0,OR-PR-151450.1 Consumer,2024-01-02,374.5,Harris Insights & Analytics,107,3.5,38.77,9.20,United States,NaN,Invoiced,NaN,NaN,Blender
1,19411.0,Non-users of all tobacco/nicotine - UK,2024-01-08,8008.0,"Povaddo, LLC",1001,8.0,47.78,7.52,United Kingdom,"Geo,Gen Pop",Invoiced,NaN,NaN,One Stop Shopper
2,19411.0,Non-users of all tobacco/nicotine - Japan,2024-01-08,8008.0,"Povaddo, LLC",1001,8.0,81.98,6.53,Japan,"Geo,Gen Pop",Invoiced,NaN,NaN,One Stop Shopper


In [6]:

df.isnull().sum()


,0
Project Code Parent,1905
Project Name,1905
Invoiced Date,1905
Actual Project Revenue,0
Client Name,0
Complete,0
CPI,0
IR,0
LOI,0
Countries,1905


In [7]:
#removed revenue
columns = ["Complete", "CPI", "IR", "LOI","Status"]

In [8]:
data = df[columns]

In [9]:
# status_dummies = pd.get_dummies(data['Status'], prefix='Status')
# data = pd.concat([data, status_dummies], axis=1)
# data.drop('Status', axis=1, inplace=True)
encoded_data = pd.get_dummies(data, columns=['Status'], prefix=['Bid'])

In [10]:
encoded_data.head()

,Complete,CPI,IR,LOI,Bid_Invoiced,Bid_Lost
0,107,3.5,38.77,9.20,True,False
1,1001,8.0,47.78,7.52,True,False
2,1001,8.0,81.98,6.53,True,False
3,1001,9.0,48.57,6.75,True,False
4,1010,9.0,38.26,9.95,True,False


In [11]:
def invoice_status(status):
       if status == 'Invoiced':
           return 1  # Invoiced
       else:
           return 0  # lost


In [12]:
data['InvoiceStatus'] = data['Status'].apply(invoice_status)

<ipython-input-12-5c3b8c736305>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['InvoiceStatus'] = data['Status'].apply(invoice_status)


In [13]:
data.drop('Status', axis=1, inplace=True)

<ipython-input-13-ef84b83262c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('Status', axis=1, inplace=True)


In [14]:
data.head()

,Complete,CPI,IR,LOI,InvoiceStatus
0,107,3.5,38.77,9.20,1
1,1001,8.0,47.78,7.52,1
2,1001,8.0,81.98,6.53,1
3,1001,9.0,48.57,6.75,1
4,1010,9.0,38.26,9.95,1


In [15]:
data.dtypes

,0
Complete,int64
CPI,float64
IR,float64
LOI,float64
InvoiceStatus,int64


In [16]:


import plotly.express as px

fig = px.box(data, y="CPI")
fig.show()


In [17]:
data[data['CPI']>data['CPI'].quantile(0.75)]['InvoiceStatus'].value_counts()

,count
InvoiceStatus,
0,587
1,2


In [18]:
data[(data['CPI']>data['CPI'].quantile(0.75)) & (data['InvoiceStatus']== 1)]

,Complete,CPI,IR,LOI,InvoiceStatus
131,12,31.33,0.33,11.83,1
252,65,32.52,0.95,16.00,1


In [19]:
data[(data['CPI'].quantile(0.50)<data['CPI']) & (data['CPI']< data['CPI'].quantile(0.75))]['InvoiceStatus'].value_counts()

,count
InvoiceStatus,
0,372
1,220


In [20]:
data.groupby('InvoiceStatus')['CPI'].median()

,CPI
InvoiceStatus,
0,9.25
1,8.00


In [21]:
data.groupby('InvoiceStatus').mean()

,Complete,CPI,IR,LOI
InvoiceStatus,,,,
0,467.494488,19.827192,36.566772,15.139213
1,489.243714,9.853617,34.078298,14.302050


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [23]:
# Assuming your data is in a DataFrame called 'data'
features = ['Complete', 'IR', 'LOI']
target = 'CPI'
# Filter data for training (InvoiceStatus == 1)
train_data = data[data['InvoiceStatus'] == 1]
X_train = train_data[features]
y_train = train_data[target]

# Data for prediction (InvoiceStatus == 0)
predict_data = data[data['InvoiceStatus'] == 0]
X_predict = predict_data[features]

In [24]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [25]:
predicted_cpi = model.predict(X_predict)
predict_data['Predicted_CPI'] = predicted_cpi

<ipython-input-25-ab1b3ae0700f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
predict_data['CPI_Difference'] = predict_data['CPI'] - predict_data['Predicted_CPI']

<ipython-input-26-ee64bc690da0>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
predict_data

,Complete,CPI,IR,LOI,InvoiceStatus,Predicted_CPI,CPI_Difference
517,50,35.00,8.0,20.0,0,19.2955,15.7045
518,20,40.00,4.0,20.0,0,22.3775,17.6225
519,40,42.00,2.0,20.0,0,18.1468,23.8532
520,10,40.00,5.0,20.0,0,21.1050,18.8950
521,15,40.00,2.0,20.0,0,22.6139,17.3861
...,...,...,...,...,...,...,...
2417,750,4.50,60.0,10.0,0,7.8924,-3.3924
2418,800,5.75,60.0,15.0,0,5.4091,0.3409
2419,100,35.00,5.0,15.0,0,16.4140,18.5860
2420,150,18.00,10.0,15.0,0,11.6879,6.3121


In [40]:
#evaluation
eval_data = data[data['InvoiceStatus'] == 1]
eva_data = eval_data[features]
eval_cpi = model.predict(eva_data)
eval_data['Predicted_CPI'] = eval_cpi

<ipython-input-40-1ed3d92ef3cc>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
eval_data['CPI_Difference'] = eval_data['CPI'] - eval_data['Predicted_CPI']

<ipython-input-41-22e57aa86ca0>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
eval_data

,Complete,CPI,IR,LOI,InvoiceStatus,Predicted_CPI,CPI_Difference
0,107,3.50,38.77,9.20,1,4.2864,-0.7864
1,1001,8.00,47.78,7.52,1,8.7329,-0.7329
2,1001,8.00,81.98,6.53,1,6.7946,1.2054
3,1001,9.00,48.57,6.75,1,9.1586,-0.1586
4,1010,9.00,38.26,9.95,1,8.2799,0.7201
...,...,...,...,...,...,...,...
512,222,7.76,11.70,7.63,1,9.3896,-1.6296
513,1007,7.90,23.64,11.04,1,7.6047,0.2953
514,1000,7.00,52.69,13.25,1,6.7277,0.2723
515,193,18.00,17.69,19.99,1,15.3004,2.6996


In [43]:
from sklearn.metrics import mean_squared_error, r2_score
mean_squared_error(eval_data['CPI'], eval_data['Predicted_CPI'])


2.6312228784526126

In [44]:

r2 = r2_score(eval_data['CPI'], eval_data['Predicted_CPI'])

In [45]:
r2

0.9359648851121832

In [46]:
## finetuning
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [60]:
# Assuming your training data is in a DataFrame called 'train_data'
features = ['Complete', 'IR', 'LOI']
target = 'CPI'
X = train_data[features]
y = train_data[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# # Fit the scaler to the training data and transform
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)

In [62]:
model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50,100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_samples': [0.5, 0.75],
    'max_depth': [None, 5, 10,20,25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1,2,3]
}

In [63]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


810 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/ut

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None, 5, 10, 20, 25],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'max_samples': [0.5, 0.75],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [64]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 17.028570026702116
R-squared: 0.6662317652402486


In [67]:

import pickle


with open('rf_predict.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [49]:
def train_and_evaluate(model, X_train, y_train, X_val, y_val, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R-squared: {r2}")
    return model, y_pred

### Logistic regression

In [50]:

# from sklearn.linear_model import LogisticRegression


# logreg_model = LogisticRegression(random_state=42,max_iter=1000)

In [51]:
# px.bar(data, y='CPI')

In [52]:

# logreg_param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
#     'penalty': ['l1', 'l2'],  # Regularization type
#     'solver': ['liblinear', 'saga'] # Solvers compatible with l1 and l2 penalties
# }

# logreg_grid_search = GridSearchCV(estimator=logreg_model, param_grid=logreg_param_grid, cv=3, scoring='accuracy') # Use appropriate scoring metric
# logreg_grid_search.fit(X_train, y_train)

# best_logreg_model = logreg_grid_search.best_estimator_
# y_pred_logreg = best_logreg_model.predict(X_val)

# #Evaluate the model
# logreg_mse = mean_squared_error(y_val, y_pred_logreg)
# logreg_r2 = r2_score(y_val, y_pred_logreg)
# print(f"Logistic Regression - Mean Squared Error: {logreg_mse}")
# print(f"Logistic Regression - R-squared: {logreg_r2}")

# print("Best hyperparameters for Logistic Regression:", logreg_grid_search.best_params_)


In [53]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression  # Import LinearRegression instead of LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [54]:

features = ['Complete', 'IR', 'LOI']
target = 'CPI'
X = train_data[features]
y = train_data[target]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Random Forest Regressor
rf_model, rf_predictions = train_and_evaluate(RandomForestRegressor(random_state=42), X_train, y_train, X_val, y_val, "Random Forest")

# 2. Linear Regression (Instead of Logistic Regression)
lr_model, lr_predictions = train_and_evaluate(LinearRegression(), X_train, y_train, X_val, y_val, "Linear Regression")  # Use LinearRegression

# 3. Decision Tree Regressor
dt_model, dt_predictions = train_and_evaluate(DecisionTreeRegressor(random_state=42), X_train, y_train, X_val, y_val, "Decision Tree")


# Comparison and Insights

# Example: Compare predictions against actual and average CPI
comparison_df = pd.DataFrame({'Actual CPI': y_val,
                             'Random Forest Predictions': rf_predictions,
                             'Logistic Regression Predictions': lr_predictions,
                             'Decision Tree Predictions': dt_predictions})

comparison_df['Average CPI'] = y_val.mean()  # Add average CPI for comparison


# Further analysis (add more comparisons/plots as needed)
# Calculate absolute errors and other metrics
comparison_df['RF_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Random Forest Predictions'])
comparison_df['LR_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Logistic Regression Predictions'])
comparison_df['DT_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Decision Tree Predictions'])

print(comparison_df.head())



Random Forest - Mean Squared Error: 18.98343540413461
Random Forest - R-squared: 0.6279154553448509
Linear Regression - Mean Squared Error: 26.870383862985552
Linear Regression - R-squared: 0.47332743881592376
Decision Tree - Mean Squared Error: 44.42076826923077
Decision Tree - R-squared: 0.12933138903358088
     Actual CPI  Random Forest Predictions  Logistic Regression Predictions  \
304       16.50                    16.4275                        12.913125   
501       24.00                    17.0562                        14.780617   
441       28.00                    21.8433                        14.203710   
153        5.67                     8.8579                        10.897402   
503       12.00                    13.3281                        12.648114   

     Decision Tree Predictions  Average CPI  RF_Abs_Error  LR_Abs_Error  \
304                      16.50    10.219904        0.0725      3.586875   
501                      11.00    10.219904        6.9438      9

In [63]:

import pandas as pd
from sklearn.preprocessing import StandardScaler
# ... (Your existing code for data loading and preprocessing) ...

# --- Model Training and Evaluation ---

# Prepare the data for all models
features = ['Complete', 'IR', 'LOI']
target = 'CPI'
X = train_data[features]
y = train_data[target]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def train_and_evaluate(model, X_train, y_train, X_val, y_val, param_grid, model_name):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R-squared: {r2}")
    print(f"{model_name} - Best Hyperparameters: {grid_search.best_params_}")
    return best_model, y_pred


# 1. Random Forest Regressor
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_model, rf_predictions = train_and_evaluate(RandomForestRegressor(random_state=42), X_train, y_train, X_val, y_val, rf_param_grid, "Random Forest")
# 2. Linear Regression
lr_param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False] ,

}
lr_model, lr_predictions = train_and_evaluate(LinearRegression(), X_train, y_train, X_val, y_val, lr_param_grid, "Linear Regression")

# 3. Decision Tree Regressor
dt_param_grid = {
    'max_depth': [None, 10, 20, 30,50,100],
    'splitter': ['best', 'random'],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 0.5],
    'ccp_alpha': [0.0, 0.1, 0.2]
}
dt_model, dt_predictions = train_and_evaluate(DecisionTreeRegressor(random_state=42), X_train, y_train, X_val, y_val, dt_param_grid, "Decision Tree")

#Comparison and Insights
comparison_df = pd.DataFrame({'Actual CPI': y_val,
                             'Random Forest Predictions': rf_predictions,
                             'Linear Regression Predictions': lr_predictions,
                             'Decision Tree Predictions': dt_predictions})

comparison_df['Average CPI'] = y_val.mean()

comparison_df['RF_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Random Forest Predictions'])
comparison_df['LR_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Linear Regression Predictions'])
comparison_df['DT_Abs_Error'] = abs(comparison_df['Actual CPI'] - comparison_df['Decision Tree Predictions'])

print(comparison_df.head())


Random Forest - Mean Squared Error: 17.54173537065881
Random Forest - R-squared: 0.6561734755116496
Random Forest - Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Linear Regression - Mean Squared Error: 26.87038386298554
Linear Regression - R-squared: 0.473327438815924
Linear Regression - Best Hyperparameters: {'copy_X': True, 'fit_intercept': True}
Decision Tree - Mean Squared Error: 17.921132819305903
Decision Tree - R-squared: 0.6487371014350991
Decision Tree - Best Hyperparameters: {'ccp_alpha': 0.2, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'splitter': 'random'}
     Actual CPI  Random Forest Predictions  Linear Regression Predictions  \
304       16.50                  16.394084                      12.913125   
501       24.00                  17.925865                      14.780617   
441       28.00                  21.930587                      14.203710   
153        5.67                  

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/uti

In [97]:

import pandas as pd
import numpy as np

def predict_cpi(complete, ir, loi, df):
    """Predicts CPI using multiple models and finds the nearest match in actual data.
    Args:
        complete: Numerical input for the 'Complete' feature.
        ir: Numerical input for the 'IR' feature.
        loi: Numerical input for the 'LOI' feature.
        df: DataFrame containing the actual CPI data.
    Returns:
        A dictionary containing the predicted CPI values from different models,
        the average predicted CPI, and the nearest match from the actual data.
    """
    exact_match = df[
        (df['Complete'] == complete) &
        (df['IR'] == ir) &
        (df['LOI'] == loi)
    ]

    if not exact_match.empty:
        nearest_match = exact_match.iloc[0]  # Take the first exact match if found
    else:
        # If no exact match, find the nearest match based on individual feature differences
        df['Complete_Diff'] = abs(df['Complete'] - complete)
        df['IR_Diff'] = abs(df['IR'] - ir)
        df['LOI_Diff'] = abs(df['LOI'] - loi)
        df['Max_Diff'] = df[['Complete_Diff', 'IR_Diff', 'LOI_Diff']].max(axis=1)
        nearest_match = df.sort_values(by=['Max_Diff']).iloc[0]

    complete,ir,loi = scaler.transform([[complete,ir,loi]])[0]
    # Input validation (example: ensure inputs are numerical and within reasonable bounds)
    if not all(isinstance(x, (int, float)) for x in [complete, ir, loi]):
        raise ValueError("Input features must be numerical.")

    # Feature scaling if applicable (consider using the same scaling as used during training)

    # Predictions using trained models (replace with your actual models)
    rf_pred = rf_model.predict(np.array([[complete, ir, loi]]))[0]
    lr_pred = lr_model.predict(np.array([[complete, ir, loi]]))[0]
    dt_pred = dt_model.predict(np.array([[complete, ir, loi]]))[0]

    # Average prediction
    avg_pred = np.mean([rf_pred, lr_pred, dt_pred])

# Attempt to find an exact match




    results = {
        "Random Forest": rf_pred,
        "Linear Regression": lr_pred,
        "Decision Tree": dt_pred,
        "Average": avg_pred,
        "Nearest Match": nearest_match.to_dict(),
    }

    return results


In [98]:
data.head()

,Complete,CPI,IR,LOI,InvoiceStatus,Difference,Complete_Diff,IR_Diff,LOI_Diff,Max_Diff
0,107,3.5,38.77,9.20,1,114.986479,107.838387,38.565975,10.268303,107.838387
1,1001,8.0,47.78,7.52,1,1003.004181,1001.838387,47.575975,8.588303,1001.838387
2,1001,8.0,81.98,6.53,1,1005.199084,1001.838387,81.775975,7.598303,1001.838387
3,1001,9.0,48.57,6.75,1,1003.035666,1001.838387,48.365975,7.818303,1001.838387
4,1010,9.0,38.26,9.95,1,1011.614504,1010.838387,38.055975,11.018303,1010.838387


In [100]:
predict_cpi(107,38,9.20,data)

<ipython-input-97-fafcbccd98de>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-97-fafcbccd98de>:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-97-fafcbccd98de>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-97-fafc

{'Random Forest': np.float64(4.914377777777777),
 'Linear Regression': np.float64(9.99856733964972),
 'Decision Tree': np.float64(7.9590000000000005),
 'Average': np.float64(7.6239817058091655),
 'Nearest Match': {'Complete': 107.0,
  'CPI': 3.5,
  'IR': 38.77,
  'LOI': 9.2,
  'InvoiceStatus': 1.0,
  'Difference': 114.98647873171363,
  'Complete_Diff': 0.0,
  'IR_Diff': 0.7700000000000031,
  'LOI_Diff': 0.0,
  'Max_Diff': 0.7700000000000031}}

In [72]:
predict_cpi(1001,82,6.5,data)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

<ipython-input-69-53c58b15d867>:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



{'Random Forest': np.float64(5.568925),
 'Linear Regression': np.float64(2.580851668995252),
 'Decision Tree': np.float64(5.053015873015872),
 'Average': np.float64(4.400930847337041),
 'Nearest Match': {'Complete': 700.0,
  'CPI': 4.4,
  'IR': 75.0,
  'LOI': 15.0,
  'InvoiceStatus': 0.0,
  'Difference': 0.0009308473370408521}}

In [59]:
import pickle

models  = ['rf_pred', 'lr_pred', 'dt_pred']
for i in models:
    with open(f'{i}.pkl', 'wb') as file:
        pickle.dump(i, file)


In [24]:
data.head()

,Complete,CPI,IR,LOI,InvoiceStatus
0,107,3.5,38.77,9.20,1
1,1001,8.0,47.78,7.52,1
2,1001,8.0,81.98,6.53,1
3,1001,9.0,48.57,6.75,1
4,1010,9.0,38.26,9.95,1


In [25]:
df = data[['Complete','CPI','IR','LOI','InvoiceStatus']]

In [26]:
from sklearn.model_selection import train_test_split

features = ['Complete','CPI','IR','LOI']
target = 'InvoiceStatus'
X = df[features]
y = df[target]

# Split data into training, testing, and evaluation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [27]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(1695, 4)
(364, 4)
(363, 4)


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [31]:
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50,100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_samples': [0.5, 0.75],
    'max_depth': [None, 5, 10,20,25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1,2,3]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


810 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
810 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/ut

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 5, 10, 20, 25],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'max_samples': [0.5, 0.75],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [32]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.03305785123966942
R-squared: 0.8002934164679993


In [81]:


y_pred_ = best_model.predict(X_val)

y_pred_


array([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,

In [82]:
mse = mean_squared_error(y_val, y_pred_)
r2 = r2_score(y_val, y_pred_)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.03305785123966942
R-squared: 0.8002934164679993


In [33]:
y_pred_proba = best_model.predict_proba(X_val)[:, 1]
y_pred_proba

array([0.82, 0.  , 0.14, 0.  , 0.  , 0.04, 0.04, 0.22, 1.  , 0.04, 0.1 ,
       0.  , 0.04, 0.04, 0.  , 0.76, 0.02, 0.72, 0.82, 0.  , 0.06, 0.36,
       0.02, 0.  , 0.  , 0.02, 0.96, 0.76, 0.14, 0.18, 0.3 , 0.02, 0.  ,
       0.92, 0.88, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.24, 0.  ,
       0.3 , 0.08, 0.02, 0.84, 0.28, 0.  , 0.36, 0.64, 0.02, 0.  , 0.  ,
       0.06, 0.  , 0.  , 0.62, 0.08, 0.86, 0.14, 0.36, 0.  , 0.  , 0.  ,
       0.84, 0.12, 0.  , 0.14, 0.74, 0.  , 0.  , 0.  , 0.  , 0.04, 0.02,
       0.04, 0.  , 0.2 , 0.04, 0.  , 0.02, 0.08, 0.  , 0.  , 0.  , 0.92,
       0.72, 0.18, 0.14, 0.12, 0.78, 0.  , 0.94, 0.  , 0.04, 0.  , 0.  ,
       0.  , 0.  , 0.02, 0.  , 0.16, 0.1 , 0.08, 0.04, 0.02, 0.02, 0.  ,
       0.92, 0.02, 0.  , 0.  , 0.02, 0.06, 0.26, 0.06, 0.  , 0.  , 0.04,
       0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.02, 0.04, 0.  , 0.06, 0.  ,
       0.34, 0.14, 0.  , 0.  , 0.3 , 0.  , 0.02, 0.  , 0.02, 0.  , 0.64,
       0.94, 0.  , 0.  , 0.4 , 0.2 , 0.  , 0.  , 0.

In [34]:
data.head(1)

,Complete,CPI,IR,LOI,InvoiceStatus
0,107,3.5,38.77,9.2,1


In [37]:
data.tail(1)

,Complete,CPI,IR,LOI,InvoiceStatus
2421,300,40.0,30.0,15.0,0


In [35]:
best_model.predict_proba([[1001,8.0,47.78,7.52]])

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



array([[0.18, 0.82]])

In [42]:
best_model.predict_proba([[300,40.0,30.0,15.0]])[0][1]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



np.float64(0.0)

In [39]:


import pickle


with open('rf_model_probab.pkl', 'wb') as file:
    pickle.dump(best_model, file)

# To load and use the model later:
# with open('best_model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)

# Example usage:
# y_pred_proba_loaded = loaded_model.predict_proba([[1001, 8.0, 47.78, 7.52]])[:, 1]
# print(y_pred_proba_loaded)
